In [1]:
import pandas as pd
import numpy as np
import os
import joblib as joblib

from tqdm.auto import tqdm
from datetime import datetime

from factorlib.utils.system import get_data_dir

tqdm.pandas()

In [106]:
raw_data_dir = get_data_dir() / 'raw'
div_season_raw = pd.read_csv(raw_data_dir / 'div_season_raw.csv')
div_season_raw.rename(columns={'PERMNO': 'permno',
                               'DISTCD': 'dstcd',
                               'DIVAMT': 'divamt',
                               'EXDT': 'exdt'}, inplace=True)
div_season_raw['exdt'] = pd.to_datetime(div_season_raw['exdt'])
div_season_raw['date'] = div_season_raw['exdt'] + pd.offsets.MonthEnd(0)
div_season_raw

,permno,dstcd,divamt,exdt,date
0,10000,NaN,NaN,NaT,NaT
1,10001,1232.0,0.04,2005-10-12,2005-10-31
2,10001,1232.0,0.05,2006-01-20,2006-01-31
3,10001,1232.0,0.08,2006-05-23,2006-05-31
4,10001,1232.0,0.10,2006-08-22,2006-08-31
...,...,...,...,...,...
386405,93432,NaN,NaN,NaT,NaT
386406,93433,NaN,NaN,NaT,NaT
386407,93434,NaN,NaN,NaT,NaT
386408,93435,NaN,NaN,NaT,NaT


In [107]:
permnos = pd.read_csv(raw_data_dir / 'permno.csv', index_col=1)
permnos.index.name = 'permno'
permnos.reset_index(inplace=True)
permnos = dict(zip(permnos['permno'], permnos['ticker']))
permnos

{10104: 'ORCL',
 10107: 'MSFT',
 10137: 'AYE',
 10138: 'TROW',
 10145: 'HON',
 10201: 'AT',
 10299: 'LLTC',
 10324: 'GENZ',
 10353: 'SXCL',
 10516: 'ADM',
 10693: 'EHC',
 10866: 'CAL',
 10874: 'BC',
 10890: 'UIS',
 10909: 'CERN',
 10942: 'CMVT',
 11081: 'DELL.1',
 11308: 'KO',
 11403: 'CDNS',
 11404: 'ED',
 11415: 'BRL',
 11552: 'CELG',
 11600: 'XRAY',
 11618: 'FAST',
 11674: 'DTE',
 11850: 'XOM',
 11891: 'MGM',
 11896: 'MXIM',
 11955: 'WM',
 11976: 'BMC',
 12052: 'GD',
 12060: 'GE',
 12062: 'LH',
 12073: 'PBCT',
 12076: 'SIG',
 12079: 'MTLQQ',
 12084: 'NXPI',
 12140: 'GR',
 12308: 'CHTR',
 12345: 'LYB',
 12346: 'LYB.B',
 12369: 'GM',
 12431: 'TT',
 12449: 'FLT',
 12458: 'RYI.1',
 12476: 'TRGP',
 12490: 'IBM',
 12503: 'NAV',
 12523: 'GNT',
 12542: 'NLSN',
 12558: 'KMI',
 12570: 'ITT',
 12622: 'HCA',
 12623: 'HII',
 12641: 'AL',
 12650: 'KSU',
 12787: 'FSL',
 12872: 'MPC',
 13035: 'XYL',
 13103: 'APTV',
 13141: 'WPX',
 13142: 'CPRI',
 13168: 'TRIP',
 13210: 'EPAM',
 13299: 'NSM',
 13323

In [108]:
with_tickers = div_season_raw.copy(deep=True)
with_tickers['ticker'] = with_tickers['permno'].apply(lambda x: permnos.get(x, np.nan)).dropna()
with_tickers.dropna(subset=['ticker'], inplace=True)
with_tickers.drop(columns=['permno'], inplace=True)
with_tickers.set_index(['ticker', 'date'], inplace=True)
with_tickers[with_tickers.index.get_level_values('ticker') == 'AMZN']

,,dstcd,divamt,exdt
ticker,date,,,
AMZN,NaT,NaN,NaN,NaT


In [109]:
with open('../../data/raw/sp500_candidates.pkl', 'rb') as p:
    candidates = joblib.load(p)
candidates

{datetime.datetime(1996, 1, 1, 0, 0): ['AAL',
  'AAMRQ',
  'AAPL',
  'ABT',
  'ACV',
  'ADM',
  'ADP',
  'ADSK',
  'AEE',
  'AEP',
  'AET',
  'AGN',
  'AIG',
  'AIT',
  'AL',
  'ALL',
  'AM',
  'AMAT',
  'AMD',
  'AMGN',
  'AMH',
  'AMP',
  'AN',
  'ANDW',
  'APD',
  'AR',
  'ARC',
  'ARNC',
  'ASC',
  'ASH',
  'AT',
  'ATI',
  'AVP',
  'AVY',
  'AXP',
  'BA',
  'BAC',
  'BAX',
  'BC',
  'BCO',
  'BCR',
  'BDK',
  'BDX',
  'BEAM',
  'BFI',
  'BFO',
  'BK',
  'BLS',
  'BLY',
  'BMET',
  'BMS',
  'BMY',
  'BNI',
  'BNL',
  'BOL',
  'BR',
  'BSX',
  'BUD',
  'C',
  'CA',
  'CAG',
  'CAL',
  'CAR',
  'CAT',
  'CB',
  'CBB',
  'CBE',
  'CCB',
  'CCI',
  'CCK',
  'CCTYQ',
  'CEG',
  'CEN',
  'CG',
  'CHA',
  'CHRS',
  'CI',
  'CL',
  'CLX',
  'CMA',
  'CMCSA',
  'CMI',
  'CNP',
  'CNW',
  'COMS',
  'COP',
  'COST',
  'COV',
  'CPB',
  'CR',
  'CSCO',
  'CSR',
  'CSX',
  'CTB',
  'CVS',
  'CVX',
  'D',
  'DALRQ',
  'DCNAQ',
  'DD',
  'DDS',
  'DE',
  'DIS',
  'DJ',
  'DLX',
  'DOV',
  'DOW',


In [110]:
all_candidates = []
for date, candidate in candidates.items():
    all_candidates.extend(candidate)
all_candidates = np.unique(all_candidates).tolist()
all_candidates

['A',
 'AABA',
 'AAL',
 'AAMRQ',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABKFQ',
 'ABMD',
 'ABT',
 'ACAS',
 'ACGL',
 'ACN',
 'ACS',
 'ACV',
 'ADBE',
 'ADCT',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'ADT',
 'AEE',
 'AEP',
 'AES',
 'AET',
 'AFL',
 'AGN',
 'AIG',
 'AIT',
 'AIV',
 'AIZ',
 'AJG',
 'AKAM',
 'AKS',
 'AL',
 'ALB',
 'ALGN',
 'ALK',
 'ALL',
 'ALLE',
 'ALTR',
 'ALXN',
 'AM',
 'AMAT',
 'AMCC',
 'AMCR',
 'AMD',
 'AME',
 'AMG',
 'AMGN',
 'AMH',
 'AMP',
 'AMT',
 'AMZN',
 'AN',
 'ANDV',
 'ANDW',
 'ANET',
 'ANF',
 'ANRZQ',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APC',
 'APD',
 'APH',
 'APOL',
 'APTV',
 'AR',
 'ARC',
 'ARE',
 'ARG',
 'ARNC',
 'ASC',
 'ASH',
 'ASND',
 'ASO',
 'AT',
 'ATGE',
 'ATI',
 'ATO',
 'ATVI',
 'AVB',
 'AVGO',
 'AVP',
 'AVY',
 'AW',
 'AWK',
 'AXP',
 'AYE',
 'AYI',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAX',
 'BBWI',
 'BBY',
 'BC',
 'BCO',
 'BCR',
 'BDK',
 'BDX',
 'BEAM',
 'BEN',
 'BFI',
 'BFO',
 'BG',
 'BGEN',
 'BHF',
 'BIG',
 'BIIB',
 'BIO',
 'BK',
 'BKNG',
 'BKR',
 'BLK',
 'BLS',
 '

In [111]:
correct_tickers = with_tickers.reset_index()
correct_tickers.dropna(subset=['date', 'ticker'], inplace=True)
correct_tickers.set_index(['date'], inplace=True)
correct_tickers = correct_tickers[correct_tickers['ticker'].isin(all_candidates)]
correct_tickers.set_index('ticker', append=True, inplace=True)
correct_tickers.sort_index(level=['ticker', 'date'])
len(correct_tickers.index.get_level_values('ticker').unique())

667

In [112]:
div_season_data = correct_tickers
div_season_data['cd1'] = div_season_data['dstcd'].astype(str).str[0].astype(float)
div_season_data['cd2'] = div_season_data['dstcd'].astype(str).str[1].astype(float)
div_season_data['cd3'] = div_season_data['dstcd'].astype(str).str[2].astype(float)
div_season_data

,,dstcd,divamt,exdt,cd1,cd2,cd3
date,ticker,,,,,,
2009-04-30,ORCL,1232.0,0.05,2009-04-06,1.0,2.0,3.0
2009-07-31,ORCL,1232.0,0.05,2009-07-13,1.0,2.0,3.0
2009-10-31,ORCL,1232.0,0.05,2009-10-09,1.0,2.0,3.0
2010-01-31,ORCL,1232.0,0.05,2010-01-14,1.0,2.0,3.0
2010-04-30,ORCL,1232.0,0.05,2010-04-12,1.0,2.0,3.0
...,...,...,...,...,...,...,...
2021-11-30,CBOE,1232.0,0.48,2021-11-29,1.0,2.0,3.0
2022-02-28,CBOE,1232.0,0.48,2022-02-25,1.0,2.0,3.0
2022-05-31,CBOE,1232.0,0.48,2022-05-27,1.0,2.0,3.0


In [113]:
first_filter = div_season_data[(div_season_data['cd1'] == 1) & (div_season_data['cd2'] == 2)]
first_filter.reset_index(level=['ticker', 'date'], inplace=True)
first_filter

,date,ticker,dstcd,divamt,exdt,cd1,cd2,cd3
0,2009-04-30,ORCL,1232.0,0.05,2009-04-06,1.0,2.0,3.0
1,2009-07-31,ORCL,1232.0,0.05,2009-07-13,1.0,2.0,3.0
2,2009-10-31,ORCL,1232.0,0.05,2009-10-09,1.0,2.0,3.0
3,2010-01-31,ORCL,1232.0,0.05,2010-01-14,1.0,2.0,3.0
4,2010-04-30,ORCL,1232.0,0.05,2010-04-12,1.0,2.0,3.0
...,...,...,...,...,...,...,...,...
34875,2021-11-30,CBOE,1232.0,0.48,2021-11-29,1.0,2.0,3.0
34876,2022-02-28,CBOE,1232.0,0.48,2022-02-25,1.0,2.0,3.0
34877,2022-05-31,CBOE,1232.0,0.48,2022-05-27,1.0,2.0,3.0
34878,2022-08-31,CBOE,1232.0,0.48,2022-08-30,1.0,2.0,3.0


In [119]:
summed_div_amts = first_filter
summed_div_amts = summed_div_amts.groupby(['ticker', 'cd3', 'date'])['divamt'].sum().reset_index()
summed_div_amts.sort_values(by=['ticker', 'date', 'cd3'], inplace=True)
summed_div_amts

,ticker,cd3,date,divamt
0,A,3.0,2012-03-31,0.100
1,A,3.0,2012-06-30,0.100
2,A,3.0,2012-09-30,0.100
3,A,3.0,2012-12-31,0.100
4,A,3.0,2013-03-31,0.120
...,...,...,...,...
34812,ZTS,3.0,2021-10-31,0.250
34813,ZTS,3.0,2022-01-31,0.325
34814,ZTS,3.0,2022-04-30,0.325
34815,ZTS,3.0,2022-07-31,0.325


In [121]:
no_dupes = summed_div_amts.groupby(['ticker', 'date']).first().reset_index()
no_dupes

,ticker,date,cd3,divamt
0,A,2012-03-31,3.0,0.100
1,A,2012-06-30,3.0,0.100
2,A,2012-09-30,3.0,0.100
3,A,2012-12-31,3.0,0.100
4,A,2013-03-31,3.0,0.120
...,...,...,...,...
34618,ZTS,2021-10-31,3.0,0.250
34619,ZTS,2022-01-31,3.0,0.325
34620,ZTS,2022-04-30,3.0,0.325
34621,ZTS,2022-07-31,3.0,0.325


In [126]:
results = no_dupes
results['divpaid'] = results['divamt'] > 0
results

,ticker,cd3,divamt,divpaid
date,,,,
2012-03-31,A,3.0,0.100,True
2012-06-30,A,3.0,0.100,True
2012-09-30,A,3.0,0.100,True
2012-12-31,A,3.0,0.100,True
2013-03-31,A,3.0,0.120,True
...,...,...,...,...
2021-10-31,ZTS,3.0,0.250,True
2022-01-31,ZTS,3.0,0.325,True
2022-04-30,ZTS,3.0,0.325,True


In [129]:
results_monthly = results
results_monthly = results_monthly.groupby('ticker').resample('M').ffill().reset_index(level='ticker', drop=True)
results_monthly

,ticker,cd3,divamt,divpaid
date,,,,
2012-03-31,A,3.0,0.100,True
2012-04-30,A,3.0,0.100,True
2012-05-31,A,3.0,0.100,True
2012-06-30,A,3.0,0.100,True
2012-07-31,A,3.0,0.100,True
...,...,...,...,...
2022-06-30,ZTS,3.0,0.325,True
2022-07-31,ZTS,3.0,0.325,True
2022-08-31,ZTS,3.0,0.325,True


In [130]:
div12 = results_monthly
div12['div12'] = div12.groupby('ticker')['divpaid'].rolling(window=12, min_periods=1).sum().reset_index(level=0, drop=True)

In [131]:
div_season = div12
div_season['DivSeason'] = 0
div_season.loc[div_season['div12'] > 0, 'DivSeason'] = 0
div_season

,ticker,cd3,divamt,divpaid,div12,DivSeason
date,,,,,,
2012-03-31,A,3.0,0.100,True,1.0,0
2012-04-30,A,3.0,0.100,True,2.0,0
2012-05-31,A,3.0,0.100,True,3.0,0
2012-06-30,A,3.0,0.100,True,4.0,0
2012-07-31,A,3.0,0.100,True,5.0,0
...,...,...,...,...,...,...
2022-06-30,ZTS,3.0,0.325,True,12.0,0
2022-07-31,ZTS,3.0,0.325,True,12.0,0
2022-08-31,ZTS,3.0,0.325,True,12.0,0


In [134]:
lagged = div_season
for i in range(1, 12):
    col_name = f'l{i}.divpaid'
    lagged[col_name] = lagged.groupby('ticker')['divpaid'].shift(i)
lagged

,ticker,cd3,divamt,divpaid,div12,DivSeason,l1.divpaid,l2.divpaid,l3.divpaid,l4.divpaid,l5.divpaid,l6.divpaid,l7.divpaid,l8.divpaid,l9.divpaid,l10.divpaid,l11.divpaid
date,,,,,,,,,,,,,,,,,
2012-03-31,A,3.0,0.100,True,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-04-30,A,3.0,0.100,True,2.0,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-31,A,3.0,0.100,True,3.0,0,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-06-30,A,3.0,0.100,True,4.0,0,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-07-31,A,3.0,0.100,True,5.0,0,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30,ZTS,3.0,0.325,True,12.0,0,True,True,True,True,True,True,True,True,True,True,True
2022-07-31,ZTS,3.0,0.325,True,12.0,0,True,True,True,True,True,True,True,True,True,True,True
2022-08-31,ZTS,3.0,0.325,True,12.0,0,True,True,True,True,True,True,True,True,True,True,True


In [137]:
second_filter = lagged
second_filter.loc[((second_filter['cd3'].isin([3, 0, 1])) &
               (second_filter['l2.divpaid'] | second_filter['l5.divpaid'] | second_filter['l8.divpaid'] | second_filter['l11.divpaid'])),
              'DivSeason'] = 1
second_filter

,ticker,cd3,divamt,divpaid,div12,DivSeason,l1.divpaid,l2.divpaid,l3.divpaid,l4.divpaid,l5.divpaid,l6.divpaid,l7.divpaid,l8.divpaid,l9.divpaid,l10.divpaid,l11.divpaid
date,,,,,,,,,,,,,,,,,
2012-03-31,A,3.0,0.100,True,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-04-30,A,3.0,0.100,True,2.0,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-31,A,3.0,0.100,True,3.0,1,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-06-30,A,3.0,0.100,True,4.0,1,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-07-31,A,3.0,0.100,True,5.0,1,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30,ZTS,3.0,0.325,True,12.0,1,True,True,True,True,True,True,True,True,True,True,True
2022-07-31,ZTS,3.0,0.325,True,12.0,1,True,True,True,True,True,True,True,True,True,True,True
2022-08-31,ZTS,3.0,0.325,True,12.0,1,True,True,True,True,True,True,True,True,True,True,True


In [138]:
third_filter = second_filter
third_filter.loc[(third_filter['cd3'] == 4) & (third_filter['l5.divpaid'] | third_filter['l11.divpaid']), 'DivSeason'] = 1
third_filter.loc[(third_filter['cd3'] == 5) & third_filter['l11.divpaid'], 'DivSeason'] = 1
third_filter

,ticker,cd3,divamt,divpaid,div12,DivSeason,l1.divpaid,l2.divpaid,l3.divpaid,l4.divpaid,l5.divpaid,l6.divpaid,l7.divpaid,l8.divpaid,l9.divpaid,l10.divpaid,l11.divpaid
date,,,,,,,,,,,,,,,,,
2012-03-31,A,3.0,0.100,True,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-04-30,A,3.0,0.100,True,2.0,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-31,A,3.0,0.100,True,3.0,1,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-06-30,A,3.0,0.100,True,4.0,1,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-07-31,A,3.0,0.100,True,5.0,1,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30,ZTS,3.0,0.325,True,12.0,1,True,True,True,True,True,True,True,True,True,True,True
2022-07-31,ZTS,3.0,0.325,True,12.0,1,True,True,True,True,True,True,True,True,True,True,True
2022-08-31,ZTS,3.0,0.325,True,12.0,1,True,True,True,True,True,True,True,True,True,True,True


In [139]:
final_filter = third_filter
final_filter = final_filter[final_filter['cd3'] != 2]
final_filter = final_filter[final_filter['cd3'] < 6]
final_filter

,ticker,cd3,divamt,divpaid,div12,DivSeason,l1.divpaid,l2.divpaid,l3.divpaid,l4.divpaid,l5.divpaid,l6.divpaid,l7.divpaid,l8.divpaid,l9.divpaid,l10.divpaid,l11.divpaid
date,,,,,,,,,,,,,,,,,
2012-03-31,A,3.0,0.100,True,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-04-30,A,3.0,0.100,True,2.0,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-31,A,3.0,0.100,True,3.0,1,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-06-30,A,3.0,0.100,True,4.0,1,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-07-31,A,3.0,0.100,True,5.0,1,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30,ZTS,3.0,0.325,True,12.0,1,True,True,True,True,True,True,True,True,True,True,True
2022-07-31,ZTS,3.0,0.325,True,12.0,1,True,True,True,True,True,True,True,True,True,True,True
2022-08-31,ZTS,3.0,0.325,True,12.0,1,True,True,True,True,True,True,True,True,True,True,True


In [143]:
div_season = final_filter.set_index('ticker', append=True)
div_season = div_season['DivSeason']
div_season.name = 'div_season'
div_season

date        ticker
2012-03-31  A         0
2012-04-30  A         0
2012-05-31  A         1
2012-06-30  A         1
2012-07-31  A         1
                     ..
2022-06-30  ZTS       1
2022-07-31  ZTS       1
2022-08-31  ZTS       1
2022-09-30  ZTS       1
2022-10-31  ZTS       1
Name: div_season, Length: 103717, dtype: int64

In [ ]:
di